In [1]:
# Imports
# Standard Libraries of Python
import itertools
from collections import Counter
from typing import Tuple, Union
from decimal import Decimal, ROUND_HALF_UP, getcontext
from datetime import datetime
getcontext().prec = 5
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score

# Dependencies
import pandas as pd
import numpy as np
from pandas import DataFrame, Index

# Libraries made for this Proyect
from backend.src.parse import Criteria
from backend.src import pick_numbers
file_route = 'database/files/'

In [31]:
# # Test of Tomorrow Numbers
# euromillions = Criteria()

# euromillions.define_odd_even()
# euromillions.define_low_high()
# euromillions.define_combinations_skips()
# euromillions.groups_info()
# euromillions.apply_transformation()
# euromillions.count_skips()
# euromillions.skips_for_last_12_draws()
# euromillions.get_natural_rotations()
# euromillions.get_numbers_clasification()
# euromillions.draw_skips()
# euromillions.skips_evaluation()

# euromillions.year_criterion()
# euromillions.rotation_criterion()
# euromillions.position_criterion()
# euromillions.group_criterion()
# euromillions.numbers_of_tomorrow()

# lotto = pick_numbers.Selection(euromillions)
# lotto.first_number()
# lotto.suggested_numbers()
lotto._selected_numbers = [50, 28, 21, 7, 11, 37, 24, 18, 40, 33]
print(lotto._selected_numbers)

content = itertools.combinations(lotto._selected_numbers,5)

tickets = []
for combination in content:
	tickets.append(combination)
	print(combination)

	combinations = [(1, 4), (2, 3), (3, 2), (4, 1), (5, 0), (0, 5)]
	# Crear variables predictoras (X) y la variable objetivo (y)
	X = euromillions.combinations_skips[['low', 'high'] + [f'days_since_{comb[0]}_{comb[1]}' for comb in combinations]]  # Incluimos las columnas de días
y = euromillions.combinations_skips['odd']  # Variable objetivo (puedes cambiar a otra variable si lo prefieres)

# Dividir en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear y entrenar el modelo
model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42
)
model.fit(X_train, y_train)

# Hacer predicciones
y_pred = model.predict(X_test)

# Evaluación
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

last_draw = euromillions.combinations_skips.iloc[-1]

# Crear las características de entrada para la predicción
last_features = [
    last_draw['low'],              # Característica low
    last_draw['high'],             # Característica high
    last_draw['days_since_1_4'],   # Días desde que salió la combinación (1, 4)
    last_draw['days_since_2_3'],   # Días desde que salió la combinación (2, 3)
    last_draw['days_since_3_2'],   # Días desde que salió la combinación (3, 2)
    last_draw['days_since_4_1'],   # Días desde que salió la combinación (4, 1)
    last_draw['days_since_5_0'],   # Días desde que salió la combinación (5, 0)
    last_draw['days_since_0_5']    # Días desde que salió la combinación (0, 5)
]

# Convertir las características en un DataFrame o un numpy array
last_features = np.array(last_features).reshape(1, -1)  # Reshape para tener un solo conjunto de datos

# Realizar la predicción
y_pred_next = model.predict(last_features)
print(f'For the next draw, there is a chance that the amount of odd numbers will be {y_pred_next}')

[50, 28, 21, 7, 11, 37, 24, 18, 40, 33]
(50, 28, 21, 7, 11)
(50, 28, 21, 7, 37)
(50, 28, 21, 7, 24)
(50, 28, 21, 7, 18)
(50, 28, 21, 7, 40)
(50, 28, 21, 7, 33)
(50, 28, 21, 11, 37)
(50, 28, 21, 11, 24)
(50, 28, 21, 11, 18)
(50, 28, 21, 11, 40)
(50, 28, 21, 11, 33)
(50, 28, 21, 37, 24)
(50, 28, 21, 37, 18)
(50, 28, 21, 37, 40)
(50, 28, 21, 37, 33)
(50, 28, 21, 24, 18)
(50, 28, 21, 24, 40)
(50, 28, 21, 24, 33)
(50, 28, 21, 18, 40)
(50, 28, 21, 18, 33)
(50, 28, 21, 40, 33)
(50, 28, 7, 11, 37)
(50, 28, 7, 11, 24)
(50, 28, 7, 11, 18)
(50, 28, 7, 11, 40)
(50, 28, 7, 11, 33)
(50, 28, 7, 37, 24)
(50, 28, 7, 37, 18)
(50, 28, 7, 37, 40)
(50, 28, 7, 37, 33)
(50, 28, 7, 24, 18)
(50, 28, 7, 24, 40)
(50, 28, 7, 24, 33)
(50, 28, 7, 18, 40)
(50, 28, 7, 18, 33)
(50, 28, 7, 40, 33)
(50, 28, 11, 37, 24)
(50, 28, 11, 37, 18)
(50, 28, 11, 37, 40)
(50, 28, 11, 37, 33)
(50, 28, 11, 24, 18)
(50, 28, 11, 24, 40)
(50, 28, 11, 24, 33)
(50, 28, 11, 18, 40)
(50, 28, 11, 18, 33)
(50, 28, 11, 40, 33)
(50, 28, 37, 24

/home/kjroy93/.conda/envs/python_main_3.11/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [18]:
euromillions.not_recommended_numbers

,numbers,criteria
0,18,2.66
1,13,2.65
2,50,2.63
3,16,2.62
4,8,2.61
5,3,2.61
6,1,2.60
7,6,2.60
8,2,2.59
9,35,2.56


In [41]:
euromillions.df.iloc[-10:]

,dates,draw,nro1,nro2,nro3,nro4,nro5,star_1,star_2,odd,even,low,high
1815,2025-02-14,1816,4,14,31,36,38,3,10,1,4,2,3
1816,2025-02-18,1817,5,14,25,26,40,5,7,2,3,3,2
1817,2025-02-21,1818,1,3,11,20,43,2,11,4,1,4,1
1818,2025-02-25,1819,17,22,23,24,44,6,11,2,3,4,1
1819,2025-02-28,1820,7,16,31,43,46,5,9,3,2,2,3
1820,2025-03-04,1821,15,20,29,35,42,2,9,3,2,2,3
1821,2025-03-07,1822,6,10,27,32,48,3,8,1,4,2,3
1822,2025-03-11,1823,13,36,37,42,47,7,11,3,2,1,4
1823,2025-03-14,1824,8,10,33,35,49,2,9,3,2,2,3
1824,2025-03-18,1825,4,9,35,46,50,8,11,2,3,2,3


In [42]:
picking_low = []
for n, ticket in enumerate(tickets):
    value = filter(lambda x: x <= 25, ticket)
    amount = sum(1 for x in list(value) if x > 0)
    if amount == 3:
        picking_low.append(n)

picking_odd = []
for n, ticket in enumerate(tickets):
    value = filter(lambda x: x % 2 != 0, ticket)
    amount = sum(1 for x in list(value) if x > 0)
    if amount == 2:
        picking_odd.append(n)

In [ ]:
euromillions.df.drop(1803)

In [ ]:
euromillions.future_groups

In [43]:
buy = []
for ticket in picking_low:
    if ticket in picking_odd:
        buy.append(tickets[ticket])

In [45]:
buy

[(50, 28, 21, 7, 24),
 (50, 28, 21, 7, 18),
 (50, 28, 21, 11, 24),
 (50, 28, 21, 11, 18),
 (50, 28, 7, 11, 24),
 (50, 28, 7, 11, 18),
 (50, 21, 7, 24, 40),
 (50, 21, 7, 18, 40),
 (50, 21, 11, 24, 40),
 (50, 21, 11, 18, 40),
 (50, 21, 37, 24, 18),
 (50, 21, 24, 18, 33),
 (50, 7, 11, 24, 40),
 (50, 7, 11, 18, 40),
 (50, 7, 37, 24, 18),
 (50, 7, 24, 18, 33),
 (50, 11, 37, 24, 18),
 (50, 11, 24, 18, 33),
 (28, 21, 7, 24, 40),
 (28, 21, 7, 18, 40),
 (28, 21, 11, 24, 40),
 (28, 21, 11, 18, 40),
 (28, 21, 37, 24, 18),
 (28, 21, 24, 18, 33),
 (28, 7, 11, 24, 40),
 (28, 7, 11, 18, 40),
 (28, 7, 37, 24, 18),
 (28, 7, 24, 18, 33),
 (28, 11, 37, 24, 18),
 (28, 11, 24, 18, 33),
 (21, 37, 24, 18, 40),
 (21, 24, 18, 40, 33),
 (7, 37, 24, 18, 40),
 (7, 24, 18, 40, 33),
 (11, 37, 24, 18, 40),
 (11, 24, 18, 40, 33)]

In [ ]:
euromillions.recommended_numbers

In [ ]:
df = pd.DataFrame(buy,columns=['1','2','3','4','5'])
df = df.iloc[:,0:6].apply(lambda x: pd.Series(sorted(x)),axis=1)
df.to_csv(f'{file_route}combinations.csv',index=False)

In [ ]:
combinations = [(1, 4), (2, 3), (3, 2), (4, 1), (5, 0), (0, 5)]
# Crear variables predictoras (X) y la variable objetivo (y)
X = euromillions.combinations_skips[['low', 'high'] + [f'days_since_{comb[0]}_{comb[1]}' for comb in combinations]]  # Incluimos las columnas de días
y = euromillions.combinations_skips['odd']  # Variable objetivo (puedes cambiar a otra variable si lo prefieres)

# Dividir en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear y entrenar el modelo
model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42
)
model.fit(X_train, y_train)

# Hacer predicciones
y_pred = model.predict(X_test)

# Evaluación
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}. Value for odd number: {y_pred}')

In [ ]:
count = 0
fails = 0
for draw in range(0,len(euromillions.df)-1):
    # Obtener el último sorteo
    last_draw = euromillions.combinations_skips.iloc[draw]

    # Crear las características de entrada para la predicción
    last_features = [
        last_draw['low'],              # Característica low
        last_draw['high'],             # Característica high
        last_draw['days_since_1_4'],   # Días desde que salió la combinación (1, 4)
        last_draw['days_since_2_3'],   # Días desde que salió la combinación (2, 3)
        last_draw['days_since_3_2'],   # Días desde que salió la combinación (3, 2)
        last_draw['days_since_4_1'],   # Días desde que salió la combinación (4, 1)
        last_draw['days_since_5_0'],   # Días desde que salió la combinación (5, 0)
        last_draw['days_since_0_5']    # Días desde que salió la combinación (0, 5)
    ]

    # Convertir las características en un DataFrame o un numpy array
    last_features = np.array(last_features).reshape(1, -1)  # Reshape para tener un solo conjunto de datos

    # Realizar la predicción
    y_pred_next = model.predict(last_features)

    # Imprimir el resultado
    next = draw + 1
    v = euromillions.combinations_skips.iloc[next,3]
    if y_pred_next == v:
        print(f'acerto el modelo con la prediccion {y_pred_next}, donde el valor {v}, se cumplio para el sorteo {next}')
        count += 1
    else:
        fails += 1
        print(f'fallo la prediccion {y_pred_next}, donde el valor {v}, no corresponde con {next}')

print(f'correctos: {count}')
print(f'Fallos: {fails}')

In [ ]:
n_predictions = len(y_test)
random_predictions = np.random.choice([0, 1, 2, 3, 4, 5], size=n_predictions, replace=True)
random_accuracy = accuracy_score(y_test, random_predictions)
random_accuracy

In [46]:
# Test of Tomorrow Stars

euromillions = Criteria(is_star=True)

euromillions.apply_transformation(is_star=True)
euromillions.count_skips(is_star=True)
euromillions.get_natural_rotations(is_star=True)
print(euromillions.year_history)

print(euromillions.aprox_rotation)
print(euromillions.exact_rotation)

      1   2   3   4   5   6   7   8   9   10  11  12
2016  18  28  15  16  18  16  16  19  20  21  20   3
2017  14  19  26  19  21  13  14  17  23  16  12  14
2018  15  18  20  21  11  16  13  19  15  11  21  28
2019  15  27  17  17  16  26  20  17  10  16  17  12
2020  16  20  20  18  15  21  17  19  16  13  15  18
2021  14  23  20  13  10  22  21  19  18  17  18  15
2022  18  17  26  12  13  20  13  16  14  14  25  20
2023  10  21  21  12  18  17  20  14  20  24  17  14
2024  21  13  19  19  16  16  18  21  19  15  10  23
2025   3   4   3   2   5   4   4   5   6   3   4   1
    hits  average_of_numbers total_average minimal_hits_needed difference
1    144            0.074844      0.084333              162.26     -18.26
2    190            0.098753      0.084333              162.26      27.74
3    187            0.097193      0.084333              162.26      24.74
4    149            0.077443      0.084333              162.26     -13.26
5    143            0.074324      0.084333     

In [81]:
euromillions.counts.iloc[-20:]

,1,2,3,4,5,6,7,8,9,10,11,12
1806,2,7,18,1,9,0,0,3,2,8,5,1
1807,3,8,19,2,0,1,1,4,0,9,6,2
1808,4,9,20,3,1,0,2,0,1,10,7,3
1809,5,10,21,0,0,1,3,1,2,11,8,4
1810,6,11,0,1,1,2,4,0,3,12,9,5
1811,0,12,1,2,2,3,5,1,0,13,10,6
1812,1,0,2,3,3,4,0,2,1,14,11,7
1813,2,1,3,4,0,5,1,3,2,0,12,8
1814,3,2,4,5,1,6,2,0,3,0,13,9
1815,0,3,5,6,2,0,3,1,4,1,14,10


In [ ]:
euromillions.df.loc[euromillions.df['dates'] == datetime(2016,9,27)]

In [51]:
def find_group(value: int) -> str | None:
    stars_group = {
    'A': [1,3,5],
    'B': [2,4,6],
    'C': [7,9,11],
    'D': [8,10,12]
}
    
    for k, v in stars_group.items():
        if value in v:
            return k
    
    return None

euromillions.df_stars['group_0'] = euromillions.df_stars['star_1'].apply(find_group) 
euromillions.df_stars['group_1'] = euromillions.df_stars['star_2'].apply(find_group)

In [ ]:
euromillions.df

In [54]:
euromillions.df_stars.iloc[-20:]

,dates,draw,star_1,star_2,group_0,group_1,combined_groups
1805,2025-01-10,1806,6,7,B,C,"[B, C]"
1806,2025-01-14,1807,5,9,A,C,"[A, C]"
1807,2025-01-17,1808,6,8,B,D,"[B, D]"
1808,2025-01-21,1809,4,5,B,A,"[B, A]"
1809,2025-01-24,1810,3,8,A,D,"[A, D]"
1810,2025-01-28,1811,1,9,A,C,"[A, C]"
1811,2025-01-31,1812,2,7,B,C,"[B, C]"
1812,2025-02-04,1813,5,10,A,D,"[A, D]"
1813,2025-02-07,1814,8,10,D,D,"[D, D]"
1814,2025-02-11,1815,1,6,A,B,"[A, B]"


In [53]:
euromillions.df_stars['combined_groups'] = euromillions.df_stars[['group_0','group_1']].apply(lambda row: [value for value in row if isinstance(value, str)], axis = 1)

In [85]:
star.iloc[-20:]

,draw,star_1,star_2,group_0,group_1,combined_groups,days_since_B_D,days_since_B_C,days_since_A_C,days_since_A_D,days_since_A_B,days_since_C_D,days_since_B_A,days_since_A_A,days_since_B_B,days_since_D_C,days_since_C_C,days_since_D_D
1805,1806,6,7,B,C,"[B, C]",1,0,2,17,23,5,26,18,16,14,29,3
1806,1807,5,9,A,C,"[A, C]",2,1,0,18,24,6,27,19,17,15,30,4
1807,1808,6,8,B,D,"[B, D]",0,2,1,19,25,7,28,20,18,16,31,5
1808,1809,4,5,B,A,"[B, A]",1,3,2,20,26,8,0,21,19,17,32,6
1809,1810,3,8,A,D,"[A, D]",2,4,3,0,27,9,1,22,20,18,33,7
1810,1811,1,9,A,C,"[A, C]",3,5,0,1,28,10,2,23,21,19,34,8
1811,1812,2,7,B,C,"[B, C]",4,0,1,2,29,11,3,24,22,20,35,9
1812,1813,5,10,A,D,"[A, D]",5,1,2,0,30,12,4,25,23,21,36,10
1813,1814,8,10,D,D,"[D, D]",6,2,3,1,31,13,5,26,24,22,37,0
1814,1815,1,6,A,B,"[A, B]",7,3,4,2,0,14,6,27,25,23,38,1


In [55]:
star = euromillions.df_stars.iloc[:,[1,2,3,4,5,6]]
# Definir las combinaciones posibles (low, high)
combinations = [('B','D'), ('B','C'), ('A','C'), ('A','D'), ('A','B'), ('C','D'), ('B','A'), ('A','A'), ('B','B'), ('D','C'), ('C','C'), ('D','D')]

# Inicializamos las nuevas columnas para almacenar los días desde la última aparición
for comb in combinations:
    col_name = f'days_since_{comb[0]}_{comb[1]}'
    star[col_name] = None

# Inicializamos el contador de días
days_since = {comb: 0 for comb in combinations}

# Iteramos sobre las filas para calcular los días desde la última aparición de cada combinación
for i in range(863,len(euromillions.df)):
    for comb in combinations:
        col_name = f'days_since_{comb[0]}_{comb[1]}'
        
        if star.loc[i, 'combined_groups'] == list(comb):
            # Si la combinación actual coincide, reiniciamos el contador a 0
            days_since[comb] = 0
        else:
            # Si no coincide, incrementamos el contador de días
            days_since[comb] += 1
        
        # Asignamos el valor del contador de días en la nueva columna
        star.loc[i, col_name] = days_since[comb]

In [62]:
avg = star.iloc[:,6:].mean().to_frame().T
last_draw = star.iloc[-1]

last_features = [
    last_draw['star_1'],
    last_draw['star_2'],
    last_draw['group_1'],              # Característica low
    last_draw['days_since_B_D'],             # Característica high
    last_draw['days_since_B_C'],   # Días desde que salió la combinación (1, 4)
    last_draw['days_since_A_C'],   # Días desde que salió la combinación (2, 3)
    last_draw['days_since_A_D'],   # Días desde que salió la combinación (3, 2)
    last_draw['days_since_A_B'],   # Días desde que salió la combinación (4, 1)
    last_draw['days_since_C_D'],   # Días desde que salió la combinación (5, 0)
    last_draw['days_since_B_A'],
    last_draw['days_since_A_A'],
    last_draw['days_since_B_B'], # Días desde que salió la combinación (0, 5)
    last_draw['days_since_D_C'],
    last_draw['days_since_C_C'],
    last_draw['days_since_D_D']
]

last_features = np.array(last_features).reshape(1, -1)
last_draw = last_draw.to_frame().T

In [ ]:
avg

In [63]:
for i in avg.columns.to_list():
    if last_draw[i].values == 0:
        print('descartado, acaba de salir')
    elif avg[i].values < last_draw[i].values:
        print(f'{i} por encima de la media, escoger esta serie')
    elif avg[i].values > last_draw[i].values and abs(avg[i].values - last_draw[i].values):
        print('preferencia')
    # elif last_draw[i].values > avg[i].values

days_since_B_D por encima de la media, escoger esta serie
preferencia
preferencia
preferencia
preferencia
days_since_C_D por encima de la media, escoger esta serie
preferencia
days_since_A_A por encima de la media, escoger esta serie
days_since_B_B por encima de la media, escoger esta serie
descartado, acaba de salir
preferencia
preferencia


In [ ]:
last_draw

In [64]:
last_draw = star.iloc[-1:]

In [65]:
star.iloc[-20:]

,draw,star_1,star_2,group_0,group_1,combined_groups,days_since_B_D,days_since_B_C,days_since_A_C,days_since_A_D,days_since_A_B,days_since_C_D,days_since_B_A,days_since_A_A,days_since_B_B,days_since_D_C,days_since_C_C,days_since_D_D
1805,1806,6,7,B,C,"[B, C]",1,0,2,17,23,5,26,18,16,14,29,3
1806,1807,5,9,A,C,"[A, C]",2,1,0,18,24,6,27,19,17,15,30,4
1807,1808,6,8,B,D,"[B, D]",0,2,1,19,25,7,28,20,18,16,31,5
1808,1809,4,5,B,A,"[B, A]",1,3,2,20,26,8,0,21,19,17,32,6
1809,1810,3,8,A,D,"[A, D]",2,4,3,0,27,9,1,22,20,18,33,7
1810,1811,1,9,A,C,"[A, C]",3,5,0,1,28,10,2,23,21,19,34,8
1811,1812,2,7,B,C,"[B, C]",4,0,1,2,29,11,3,24,22,20,35,9
1812,1813,5,10,A,D,"[A, D]",5,1,2,0,30,12,4,25,23,21,36,10
1813,1814,8,10,D,D,"[D, D]",6,2,3,1,31,13,5,26,24,22,37,0
1814,1815,1,6,A,B,"[A, B]",7,3,4,2,0,14,6,27,25,23,38,1


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Crear el DataFrame (sustituye esto por la carga de tus datos reales)
data = star.copy()  # Aquí carga el DataFrame real.

# Separar las características y la variable objetivo
X = data.drop(columns=['draw','star_1','star_2','group_1','combined_groups'])
y = data['group_0']

# Convertir las variables categóricas a numéricas si es necesario
categorical_columns = ["group_0"]
for col in categorical_columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])

# Codificar el objetivo (combined_groups)
le_target = LabelEncoder()
y = le_target.fit_transform(y)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar un modelo
model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42
)
model.fit(X_train, y_train)

# Evaluar el modelo
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
X_test

In [ ]:
categorical_columns = ['group_0', 'group_1']
for col in categorical_columns:
    le = LabelEncoder()
    star[col] = le.fit_transform(star[col])

In [ ]:
last_draw = star.iloc[-1]

last_features = [
    last_draw['star_1'],
    last_draw['star_2'],
    last_draw['group_1'],              # Característica low
    last_draw['days_since_B_D'],             # Característica high
    last_draw['days_since_B_C'],   # Días desde que salió la combinación (1, 4)
    last_draw['days_since_A_C'],   # Días desde que salió la combinación (2, 3)
    last_draw['days_since_A_D'],   # Días desde que salió la combinación (3, 2)
    last_draw['days_since_A_B'],   # Días desde que salió la combinación (4, 1)
    last_draw['days_since_C_D'],   # Días desde que salió la combinación (5, 0)
    last_draw['days_since_B_A'],
    last_draw['days_since_A_A'],
    last_draw['days_since_B_B'], # Días desde que salió la combinación (0, 5)
    last_draw['days_since_D_C'],
    last_draw['days_since_C_C'],
    last_draw['days_since_D_D']
]

last_features = np.array(last_features).reshape(1, -1)

In [ ]:
last_features

In [ ]:
y_pred_next = model.predict(last_features)

In [ ]:
star.iloc[906]

In [ ]:
count = 0
fails = 0
for draw in range(863,len(euromillions.df)-1):
    # Obtener el último sorteo
    last_draw = star.loc[draw]

    # Crear las características de entrada para la predicción
    last_features = [
        last_draw['group_1'],              # Característica low
        last_draw['days_since_B_D'],             # Característica high
        last_draw['days_since_B_C'],   # Días desde que salió la combinación (1, 4)
        last_draw['days_since_A_C'],   # Días desde que salió la combinación (2, 3)
        last_draw['days_since_A_D'],   # Días desde que salió la combinación (3, 2)
        last_draw['days_since_A_B'],   # Días desde que salió la combinación (4, 1)
        last_draw['days_since_C_D'],   # Días desde que salió la combinación (5, 0)
        last_draw['days_since_B_A'],
        last_draw['days_since_A_A'],
        last_draw['days_since_B_B'], # Días desde que salió la combinación (0, 5)
        last_draw['days_since_D_C'],
        last_draw['days_since_C_C'],
        last_draw['days_since_D_D']
    ]

    # Convertir las características en un DataFrame o un numpy array
    last_features = np.array(last_features).reshape(1, -1)  # Reshape para tener un solo conjunto de datos

    # Realizar la predicción
    y_pred_next = model.predict(last_features)

    # Imprimir el resultado
    next = draw + 1
    v = star.loc[next,'group_0']
    if y_pred_next == v:
        print(f'acerto el modelo con la prediccion {y_pred_next}, donde el valor {v}, se cumplio para el sorteo {next}')
        count += 1
    else:
        fails += 1
        print(f'fallo la prediccion {y_pred_next}, donde el valor {v}, no corresponde con {next}')

print(f'correctos: {count}')
print(f'Fallos: {fails}')

In [ ]:
euromillions.df_stars